In [1]:
from warnings import filterwarnings

filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [5]:
alliance_df = pd.read_csv('data/alliance_v4.1_by_directed_yearly.csv', encoding = 'utf8')

alliance_df.rename({'left_censor': 'pre_1816_alliance', 'right_censor': 'in_effect_1231_2012',
          'neutrality': 'neutrality_treaty', 'nonaggression': 'nonaggression_treaty',
          'defense': 'defense_treaty', 'entente': 'entente_treaty',
          'ccode1': 'c_code_1', 'ccode2': 'c_code_2', 'state_name1': 'state_name_1',
          'state_name2': 'state_name_2'}, axis = 1, inplace = True)

alliance_df['alliance_date_start'] = pd.to_datetime((alliance_df['dyad_st_year']*10000+alliance_df['dyad_st_month']*100+alliance_df['dyad_st_day']).apply(str),format='%Y%m%d')
alliance_df['alliance_date_end'] = pd.to_datetime((alliance_df['dyad_end_year']*10000+alliance_df['dyad_end_month']*100+alliance_df['dyad_end_day']).apply(str),format='%Y%m%d')

alliance_df = alliance_df.sort_values(by = 'year', ascending = True).reset_index()

alliance_df.drop(['dyad_st_day', 'dyad_st_month', 'dyad_st_year', 'dyad_end_day',
         'dyad_end_month', 'dyad_end_year', 'version4id', 'version', 'index'],
         axis = 1, inplace = True)

In [6]:
alliance_df.head()

,c_code_1,state_name_1,c_code_2,state_name_2,pre_1816_alliance,in_effect_1231_2012,defense_treaty,neutrality_treaty,nonaggression_treaty,entente_treaty,year,alliance_date_start,alliance_date_end
0,200,United Kingdom,235,Portugal,1,1,1,0,1.00,0.00,1816,1816-01-01,NaT
1,275,Hesse Grand Ducal,300,Austria-Hungary,1,0,1,0,1.00,1.00,1816,1816-01-01,1848-03-15
2,275,Hesse Grand Ducal,273,Hesse Electoral,1,0,1,0,1.00,1.00,1816,1816-01-01,1848-03-15
3,275,Hesse Grand Ducal,271,Wuerttemburg,1,0,1,0,1.00,1.00,1816,1816-01-01,1848-03-15
4,275,Hesse Grand Ducal,269,Saxony,1,0,1,0,1.00,1.00,1816,1816-01-01,1848-03-15


In [7]:
aggregations = {
  'c_code_2': 'count',
  'pre_1816_alliance': 'sum',
  'in_effect_1231_2012': 'sum',
  'defense_treaty': 'sum',
  'neutrality_treaty': 'sum',
  'nonaggression_treaty': 'sum',
  'entente_treaty': 'sum'
  }

alliance_df_group = alliance_df.groupby(['c_code_1', 'state_name_1', 'year']).agg(aggregations).reset_index()

alliance_df_group = alliance_df_group.sort_values(by = 'year', ascending = True).reset_index()

alliance_df_group.rename({'c_code_2': 'num_alliances', 'pre_1816_alliance': 'pre_1816_alliances',
             'in_effect_1231_2012': 'num_in_effect_1231_2012',
             'defense_treaty': 'defense_treaties',
             'neutrality_treaty': 'neutrality_treaties',
             'nonaggression_treaty': 'nonaggression_treaties',
             'entente_treaty': 'entente_treaties'}, axis = 1, inplace = True)

alliance_df_group.drop(['index'], axis = 1, inplace = True)

In [8]:
alliance_df_group.to_pickle('pickle/alliance_df_group.pkl')
alliance_df_group.to_csv('dataframe_exports/alliance_df_group.csv')

In [9]:
alliance_df_group.head()

,c_code_1,state_name_1,year,num_alliances,pre_1816_alliances,num_in_effect_1231_2012,defense_treaties,neutrality_treaties,nonaggression_treaties,entente_treaties
0,230,Spain,1816,1,0,0,1,0,0.00,0.00
1,255,Germany,1816,10,10,0,10,0,7.00,10.00
2,267,Baden,1816,7,7,0,7,0,7.00,7.00
3,271,Wuerttemburg,1816,7,7,0,7,0,7.00,7.00
4,380,Sweden,1816,1,1,0,0,0,0.00,1.00
